## Jimmy requested eclipse timings for TIC 4707 for a range of days during 2024Q4 so if there's a B-system eclipse we might request that particular day in our proposal.

In [37]:
import astropy.table as tb
from astropy.time import Time
import astropy.units as u


In [5]:
ephemerides = tb.vstack([
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2022 Binary Parameters.csv'),
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2023 Binary Parameters.csv'),
])
ephemerides[ephemerides["Local ID"] == "TIC 470710327"]

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 470710327,A,1.104686,1765.1668,1765.721905,60,50,5.8,5.5
TIC 470710327,B,19.950922,1805.4542,--,70,--,--,--


In [65]:
tess_offset = 2457000
beg_date = Time("2024-11-09", format="iso", scale="utc").jd - tess_offset
end_date = Time("2024-11-21", format="iso", scale="utc").jd - tess_offset

for system in ephemerides[ephemerides["Local ID"] == "TIC 470710327"]:
    for member_letter, member_name in [("a", "Primary"), ("b", "Secondary")]:
        name = f"T0 {member_name}"
        if system[name] > 0:
            t0 = system[name]
            period = system["Period"]
            if not (half_duration := system[f"Duration {member_name}"] / 24 / 2):
                half_duration = 0
            mid_eclipse = t0 + int((beg_date - t0) / period) * period
            while True:
                eclipse_beg = mid_eclipse - half_duration
                eclipse_end = mid_eclipse + half_duration
                print(system["Member"], member_letter, Time(eclipse_beg + tess_offset, format="jd").iso, Time(eclipse_end + tess_offset, format="jd").iso, t0, period)
                mid_eclipse += period
                if mid_eclipse > end_date:
                    break


A a 2024-11-08 15:04:03.533 2024-11-08 20:52:03.533 1765.1668 1.104686
A a 2024-11-09 17:34:48.403 2024-11-09 23:22:48.403 1765.1668 1.104686
A a 2024-11-10 20:05:33.274 2024-11-11 01:53:33.274 1765.1668 1.104686
A a 2024-11-11 22:36:18.144 2024-11-12 04:24:18.144 1765.1668 1.104686
A a 2024-11-13 01:07:03.014 2024-11-13 06:55:03.014 1765.1668 1.104686
A a 2024-11-14 03:37:47.885 2024-11-14 09:25:47.885 1765.1668 1.104686
A a 2024-11-15 06:08:32.755 2024-11-15 11:56:32.755 1765.1668 1.104686
A a 2024-11-16 08:39:17.626 2024-11-16 14:27:17.626 1765.1668 1.104686
A a 2024-11-17 11:10:02.496 2024-11-17 16:58:02.496 1765.1668 1.104686
A a 2024-11-18 13:40:47.366 2024-11-18 19:28:47.366 1765.1668 1.104686
A a 2024-11-19 16:11:32.237 2024-11-19 21:59:32.237 1765.1668 1.104686
A a 2024-11-20 18:42:17.107 2024-11-21 00:30:17.107 1765.1668 1.104686
A b 2024-11-08 02:01:39.734 2024-11-08 07:31:39.734 1765.721905 1.104686
A b 2024-11-09 04:32:24.605 2024-11-09 10:02:24.605 1765.721905 1.104686
A 

In [232]:
from astropy.table.row import Row
from astropy.time import Time
import astropy.units as u

def calc_eclipse_timings(
    t0: Time = None,
    period: u.Quantity = None,
    duration: u.Quantity = None,
    beg_time: Time = None,
    end_time: Time = None,
    count: int = 0,
    row = None,
) -> list[tuple[Time, Time]]:
    """Given an initial time (t0), a period and optionally a duration, return a list of eclipses.
    The results can be a fixed number of eclipses, or all eclipses falling between a beginning and end time.
    Period & duration are Astropy Quantity, t0, beg_time & end_time are Astropy Time objects, and count is an int.
    """
    # process & validate inputs
    if isinstance(row, Row):
        t0 = row["T0"]
        period = row["Period"]
        if "Duration" in row.columns:
            duration = row["Duration"]
    if beg_time is None:
        raise ValueError("Must specify begin time")
    if count > 0:
        if end_time is not None:
            raise ValueError("Cannot specify both a count and an end time")
        use_count = True
    else:
        if count < 0:
            raise ValueError("Count cannot be negative")
        if end_time is None:
            raise ValueError("If count is zero, end time must be specified")
        use_count = False


    if duration is not None:
        half_duration = duration / 2
    t = t0 + (round((beg_time.jd - t0.jd) / period.value) * period)
    while t - half_duration < beg_time:
        t += period
    eclipses = []
    while True:
        if duration is not None:
            eclipse_beg = t - half_duration
            eclipse_end = t + half_duration
        else:
            eclipse_beg = t
            eclipse_end = t
        eclipses.append((eclipse_beg, eclipse_end))
        print(eclipse_beg.iso, "   ", eclipse_end.iso)
        t += period
        if use_count:
            if len(eclipses) >= count:
                break
        else:
            if end_time < t + half_duration:
                break
    return eclipses


beg_time = Time("2024-11-10", format="iso", scale="utc")
end_time = Time("2024-11-21", format="iso", scale="utc")

foo = QTable(names=["T0", "Period", "Duration"], dtype=[Time, u.Quantity, u.Quantity])
# foo.add_row([1,2 * u.day,3])
foo.add_row([Time(1765.1668 + tess_offset, format="jd"), 1.104686 * u.day, 5.8 * u.hour])
calc_eclipse_timings(row=foo[0], beg_time=beg_time, count=1)

# print("Aa")
# calc_eclipse_timings(Time(1765.1668 + tess_offset, format="jd"), 1.104686 * u.day, 5.8 * u.hour, beg_time, end_time)
# print("Ab")
# calc_eclipse_timings(Time(1765.721905 + tess_offset, format="jd"), 1.104686 * u.day, 5.5 * u.hour, beg_time, end_time)
# print("Ba")
# calc_eclipse_timings(Time(1805.4542 + tess_offset, format="jd"), 19.950922 * u.day, None, beg_time, end_time)

2024-11-10 20:05:33.274     2024-11-11 01:53:33.274


[(<Time object: scale='utc' format='jd' value=2460625.3371906662>,
  <Time object: scale='utc' format='jd' value=2460625.578857333>)]

## After catching a bug in the one-off code (while testing the above function), do some manual cross-checking to make sure I'm not jerking Jimmy around with bad answers.

In [56]:
foo = Time("2024-11-06 11:42:52.013", format="iso", scale="utc").jd - tess_offset
(foo - 1805.4542) / 19.950922


91.00000000012707

In [58]:
19.950922 * 91 + 1805.4542 + tess_offset

2460620.988102

## Convert CSV file of Kostov ephemerides to something easier to manage, and in standard units.

In [269]:
from astropy.table import QTable

better_ephem = QTable(names=["Local ID", "System", "Member", "T0", "Period", "Duration", "Depth"],
                      units={"Period": u.day, "Duration": u.hour},
                      dtype=[str, str, str, float, u.Quantity, u.Quantity, float],
                      )
# better_ephem["Period"].unit = u.day
# better_ephem["Duration"].unit = u.hour

for ephem in ephemerides:
    for letter, word in [("a", "Primary"), ("b", "Secondary")]:
        if not ephem[f"T0 {word}"] or ephem["Period"] < 0:
            continue
        if ephem[f"Duration {word}"]:
            duration = ephem[f"Duration {word}"]# * u.hour
        else:
            duration = None
        better_ephem.add_row([
            ephem["Local ID"],
            ephem["Member"],
            letter,
            Time(ephem[f"T0 {word}"] + tess_offset, format="jd").jd,
            ephem["Period"],# * u.day,
            duration,
            ephem[f"Depth {word}"] / 1000,
            ],
            # mask = [False, False, False, False, False, not (ephem[f"Duration {word}"] > 0), False],
            )

better_ephem.write("Kostov ephemerides.csv", overwrite=True)
better_ephem

Local ID,System,Member,T0,Period,Duration,Depth
,,,,d,h,
str14,str1,str1,float64,object,object,float64
TIC 9493888,A,a,2458816.2345,2.098992,2.9,0.146
TIC 9493888,A,b,2458817.283786,2.098992,2.7,0.117
TIC 25818450,A,a,2458769.9109,10.132402,None,0.012
TIC 25818450,A,b,2458776.391584,10.132402,None,0.009
TIC 27543409,A,a,2458493.1001,2.122862,None,0.05
TIC 27543409,A,b,2458494.153889,2.122862,None,0.015
TIC 31928452,A,a,2458337.9129,2.8823,2.1,0.03
TIC 31928452,A,b,2458339.358085,2.8823,1.6,0.025


In [267]:
foo = QTable.read("Kostov ephemerides.ecsv")
foo.add_row(["mine", "A", "a", 1234.5678, 12*u.day, 14*u.day, None])
foo.write("foo.csv")

In [179]:
tics_of_interest = [
# ("2024-05-27", "TIC 344541836"),
# ("2024-05-28", "TIC 278465736"),
# ("2024-02-23", "TIC 161043618"),
# ("2024-02-24", "TIC 367448265"),
  ("", "TIC 470710327"),
]

beg_time = Time("2024-10-16", format="iso", scale="utc")
end_time = Time("2024-10-23", format="iso", scale="utc")
# beg_time = Time("2024-08-23", format="iso", scale="utc")
# end_time = Time("2024-09-11", format="iso", scale="utc")

for _, tic_of_interest in tics_of_interest:
    print()
    print(tic_of_interest)
    for ephem in better_ephem[better_ephem["Local ID"] == tic_of_interest]:
        if ephem["Duration"] is None:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration=unknown, Depth={ephem["Depth"]}")
        else:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration={ephem["Duration"]:7.4f}, Depth={ephem["Depth"]}")
        calc_eclipse_timings(ephem["T0"], ephem["Period"], ephem["Duration"], beg_time, end_time)


TIC 470710327
  Aa, Period=  1.10469 d, Duration= 5.8000 h, Depth=0.06
2024-10-16 10:18:21.254     2024-10-16 16:06:21.254
2024-10-17 12:49:06.125     2024-10-17 18:37:06.125
2024-10-18 15:19:50.995     2024-10-18 21:07:50.995
2024-10-19 17:50:35.866     2024-10-19 23:38:35.866
2024-10-20 20:21:20.736     2024-10-21 02:09:20.736
2024-10-21 22:52:05.606     2024-10-22 04:40:05.606
  Ab, Period=  1.10469 d, Duration= 5.5000 h, Depth=0.05
2024-10-15 21:15:57.456     2024-10-16 02:45:57.456
2024-10-16 23:46:42.326     2024-10-17 05:16:42.326
2024-10-18 02:17:27.197     2024-10-18 07:47:27.197
2024-10-19 04:48:12.067     2024-10-19 10:18:12.067
2024-10-20 07:18:56.938     2024-10-20 12:48:56.938
2024-10-21 09:49:41.808     2024-10-21 15:19:41.808
2024-10-22 12:20:26.678     2024-10-22 17:50:26.678
  Ba, Period= 19.95092 d, Duration=unknown, Depth=0.07
2024-10-17 12:53:32.352     2024-10-17 12:53:32.352


In [162]:
better_ephem[(better_ephem["Local ID"]=="TIC 367448265") & (better_ephem["System"]=="A") & (better_ephem["Member"]=="a")]["Duration"]

-- h
